# Explaining Tensorflow Code for a Convolutional Neural Network
<br>
from: http://www.jessicayung.com/explaining-tensorflow-code-for-a-convolutional-neural-network/

## 1. Code

In [1]:
import tensorflow as tf

# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input]) # input, i.e. pixels that constitute the image
y = tf.placeholder(tf.float32, [None, n_classes]) # labels, i.e which digit the image is
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [2]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    # Fully connected layer
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [3]:

# Store layers weight &amp; bias
weights = {
# 5x5 conv, 1 input, 32 outputs
'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
# 5x5 conv, 32 inputs, 64 outputs
'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
# fully connected, 7*7*64 inputs, 1024 outputs
'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
# 1024 inputs, 10 outputs (class prediction)
'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
'bc1': tf.Variable(tf.random_normal([32])),
'bc2': tf.Variable(tf.random_normal([64])),
'bd1': tf.Variable(tf.random_normal([1024])),
'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

NameError: name 'conv2d' is not defined

## 2. Translating the code

"Let’s draw the model the function conv_net represents. <u> The batch size given is 128. That means that each time, at most 128 images are fed into our model.</u>"

"The big picture:"

<img src='../conv-big-pic.png' style="max-width:25%;width:auto;height:auto;">


<img src="../imgs/cnn-1.png">

<img src="../imgs/cnn-2.png">

"We can see that there are fives types of layers here:
<ul>
    <li>convolution layers,
    <li>max pooling layers,
    <li>layers for reshaping input,
    <li>fully-connected layers and
    <li>dropout layers.
</ul>
"

## 2.1 What is conv2d (convolution layer)?

"A convolution layer tries to extract higher-level features by replacing data for each (one) pixel with a value computed from the pixels covered by the e.g. 5×5 filter centered on that pixel(all the pixels in that region)."

<img src="../imgs/cnn-fletcher-bach.gif" style="max-width:25%;width:auto;height:auto;">

"We slide the filter across the width and height of the input and compute the dot products between the entries of the filter and input at each position. I explain this further when discussing tf.nn.conv2d() below."

In [1]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

This function comprises three parts:

<ol> 
    <li> Conv2D layer from Tensorflow </li>
            <ol>
                <li> tf.nn.conv2d() </li>
                <li> This is analogous to xW (multiplying input by weights) in a fully connected layer. </li>
            </ol>
    <li> Add bias </li>
    <li> ReLU activation </li>
        <ol>
            <li> This transforms the output like so:ReLU(x) = max(0,x). (See previous post for more details). </li>
        </ol>
</ol>


<img src="../imgs/relu.png">

"You can see it is structurally the same as a fully connected layer, except we multiply the input with weights in a different way."

### Conv2D layer

"The key part here is tf.nn.conv2d(). Let’s look at each of its arguments."

"x is the input.
W are the weights.

The weights have four dimensions:  [filter_height, filter_width, input_depth, output_depth]. What this means is that we have  output_depth filters in this layer. Each filter considers information with dimensions [filter_height, filter_width, input_depth] at a time. Yes, each filter goes through ALL the input depth layers. "

<ul>
    <li> x is the input. </li>
    <li> W are the weights </li>
        <ul>
            <li> The weights have four dimensions:  [filter_height, filter_width, input_depth, output_depth] </li>
            <li> What this means is that we have  output_depth filters in this layer. </li>
                <ul> 
                    <li> Each filter considers information with dimensions [filter_height, filter_width, input_depth] at a time. Yes, each filter goes through ALL the input depth layers. </li>
                    <li> <img src="../imgs/3d-cnn.png">
                    <li>This is like how, in a fully connected layer, we may have ten neurons, each of which interacts with all the neurons in the previous layer.</li>
    
    